In [2]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import os 
import numpy as np 
import ast



# Loading Data

In [2]:
df=pd.read_csv('beer data/Beer_data20220916.csv')
df_beers=pd.read_csv('beer_data3.csv')
df_=df.copy()


In [3]:
df_beers

,beer_name,beer_page,beer_company_name,beer_company_page_url,abv,rating,avg,last_active,beer_type_url,style_name,beer_type_name,beer_id
0,Amber,/beer/profile/429/1309/,Alaskan Brewing Co.,/beer/profile/429/,5.30,2254,3.65,09-07-2022,https://www.beeradvocate.com/beer/styles/86/,Brown Ales,Altbier,429_1309
1,Double Bag,/beer/profile/94/273/,Long Trail Brewing Co.,/beer/profile/94/,7.20,1781,3.87,08-30-2022,https://www.beeradvocate.com/beer/styles/86/,Brown Ales,Altbier,94_273
2,Long Trail Ale,/beer/profile/94/3128/,Long Trail Brewing Co.,/beer/profile/94/,5.00,1429,3.57,08-30-2022,https://www.beeradvocate.com/beer/styles/86/,Brown Ales,Altbier,94_3128
3,Doppelsticke,/beer/profile/1923/24042/,Uerige Obergärige Hausbrauerei,/beer/profile/1923/,8.50,721,4.10,08-22-2022,https://www.beeradvocate.com/beer/styles/86/,Brown Ales,Altbier,1923_24042
4,Sleigh'r Dark Doüble Alt Ale,/beer/profile/14400/47120/,Ninkasi Brewing Company,/beer/profile/14400/,7.20,625,3.76,04-11-2021,https://www.beeradvocate.com/beer/styles/86/,Brown Ales,Altbier,14400_47120
...,...,...,...,...,...,...,...,...,...,...,...,...
194115,Voodoo - U,/beer/profile/54266/489724/,Apex Aleworks,/beer/profile/54266/,8.50,0,0.00,06-15-2020,https://www.beeradvocate.com/beer/styles/48/,Wheat Beers,Witbier,54266_489724
194116,Wit,/beer/profile/57106/491260/,Rochester Brewing & Roasting Co.,/beer/profile/57106/,4.90,0,0.00,06-24-2020,https://www.beeradvocate.com/beer/styles/48/,Wheat Beers,Witbier,57106_491260
194117,Brumada,/beer/profile/60099/519676/,Wolf Branch Brewing,/beer/profile/60099/,4.80,0,0.00,11-23-2020,https://www.beeradvocate.com/beer/styles/48/,Wheat Beers,Witbier,60099_519676
194118,Lowside,/beer/profile/48685/464893/,Strap Tank Brewing Co.,/beer/profile/48685/,4.80,0,0.00,02-07-2020,https://www.beeradvocate.com/beer/styles/48/,Wheat Beers,Witbier,48685_464893


In [6]:
df['Learn more about this style.'].unique().shape

(120,)

In [13]:
df.beer_id.unique().shape

(194118,)

# Cleaning Data

## Cleaning Notes

In [14]:
df.note=df.note.str\
            .replace('\n',' ',regex=True)\
            .replace('Notes: ','',regex=True)\
            .replace('None','',regex=True)\


## Cleaning reviews

In [15]:
df.reviews=df.reviews

#str to list
df.reviews=df.reviews.apply(lambda x: ast.literal_eval(x))


### str to beautiful soup

not really efficient. Next time it would be better to preprocess the data beforehand.


In [16]:
review_list=df.reviews.values.tolist()

review_list_soup=[]
for e in tqdm(review_list):
    if len(e)==0:
        review_list_soup.append([])
    else:
        review_list_soup.append([BeautifulSoup(e2) for e2 in e])


df.reviews=review_list_soup


100%|██████████| 194118/194118 [41:05<00:00, 78.75it/s]   


### getting reviews and a bit of cleaning

In [17]:
#BAscore_norm=e.find('span',{'class':'BAscore_norm'}).text
attr={'style':'margin:10px 0px; padding:10px; border-left:2px solid #A9A9A9; font-size:11pt; line-height:1.4;'}
df['reviews_list']=df.reviews.apply(lambda x: [e.find('div',attr).text for e in x if e.find('div',attr ) is not None] )
df['reviews_list']=df.reviews_list.apply(lambda x: [e.replace('\n',' ') for e in x] )



## Cleaning the rest 

In [18]:
df.drop(columns='reviews',inplace=True)

df.rename(columns={
    'View more beers and info from this brewery.':'brewery',
    'Learn more about this style.':"beer_style",
    'Percentage of alcohol by volume.':'ABV (%)',
    'The official BeerAdvocate (BA) Score for this beer.':'BA score',
    'Average across all ratings for this beer.':"avg rating",
    'Number of reviews for this beer.':"N reviews",
    'Number of ratings for this beer.':"N ratings",
    "The beer's current activity status.":'activity status',
    'The date this beer was last rated.':"last rated date",
    'The date this beer was added to the site.':"date the beer was added",
    'Number of users who want this beer.':"N wants",
    'Number of users who have this beer.':"N had",
    'reviews_list':'reviews'
    },inplace=True)

df['ABV (%)']=df['ABV (%)'].str.replace('%','').astype(float)
df['N reviews']=df['N reviews'].str.replace(',','').astype(int)
df['N ratings']=df['N ratings'].str.replace(',','').astype(int)
df['last rated date']=df['last rated date'].str.replace('\n','')

In [19]:
df.explode('reviews').to_csv('Cleaned beer data/Beerdata_exploded.csv',index=False)
df.to_csv('Cleaned beer data/Beerdata.csv',index=False)

## cleaning location 

I will be using [Geopy](https://geopy.readthedocs.io/en/stable/index.html)



In [27]:
from geopy import Nominatim




def find_loc(locations,locs_dict={}):
    for loc in tqdm(locations):
        try:
            location = locator.geocode(loc)
            locs_dict[loc]=location
        except:
            print(F'Error: {loc}')
            pass

            #new list by removing those who have been done
            #locations=list(set(locations)-set(locs_dict.keys()))
            #find_loc(locations,locs_dict=locs_dict)
    return locs_dict 

df1=pd.read_csv('Cleaned beer data/Beerdata.csv')
#Those 2 locations did not have any matchs
df1.location=df1.location.str\
    .replace('Virgin Islands (U.S.)','Virgin Islands',regex=False)\
    .replace('England, Croatia (Hrvatska)','England',regex=False)



locator = Nominatim(user_agent="myGeocoder")
locations=df1.location.unique()

loc_dict=find_loc(locations)


df1['location_cleaned']=df1.location.apply(lambda x: loc_dict.get(x,np.NaN) ) #loc_dict
df1['address']=df1.location_cleaned.apply(lambda x: x.address if (x is not np.NaN and x is not None)  else np.NaN)
df1['loc_altitude']=df1.location_cleaned.apply(lambda x: x.altitude if (x is not np.NaN and x is not None)  else np.NaN)
df1['loc_latitude']=df1.location_cleaned.apply(lambda x: x.latitude if (x is not np.NaN and x is not None)  else np.NaN)
df1['loc_longitude']=df1.location_cleaned.apply(lambda x: x.longitude if (x is not np.NaN and x is not None)  else np.NaN)

100%|██████████| 213/213 [01:46<00:00,  2.00it/s]


In [32]:
df1.to_csv('Cleaned beer data/Beerdata.csv',index=False)